In [2]:
from typing import Callable, Literal, Any

from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [3]:
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [4]:
data = dataset
data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
data["train"]["document"][0]

'앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.'

In [6]:
print(sorted(list(set(data["train"]["document"][0]))))

[' ', '.', '0', '1', '2', '3', '4', '5', 'E', 'N', 'T', 'U', 'Y', '가', '개', '것', '겠', '격', '견', '겸', '경', '고', '공', '과', '관', '국', '규', '극', '금', '급', '기', '까', '나', '난', '너', '높', '는', '늘', '니', '다', '단', '달', '담', '당', '대', '도', '되', '될', '뒷', '들', '등', '때', '또', '라', '략', '량', '러', '려', '력', '련', '로', '록', '롯', '류', '를', '리', '린', '마', '만', '말', '면', '모', '목', '무', '물', '박', '반', '받', '방', '버', '벌', '보', '복', '본', '부', '비', '산', '상', '서', '선', '성', '세', '소', '속', '쇠', '수', '스', '습', '승', '시', '실', '악', '안', '액', '앵', '야', '양', '억', '업', '에', '엔', '여', '역', '연', '열', '였', '올', '외', '용', '우', '운', '울', '원', '월', '위', '육', '율', '융', '으', '은', '을', '응', '의', '이', '인', '임', '입', '있', '자', '장', '재', '적', '전', '정', '제', '조', '주', '줄', '중', '증', '지', '진', '참', '창', '책', '척', '첨', '체', '초', '총', '최', '추', '출', '침', '커', '케', '크', '통', '투', '특', '팀', '팅', '편', '표', '하', '한', '할', '합', '해', '했', '현', '호', '홍', '화', '확', '환', '황', '회', '획', '효', '히']


In [7]:
ko_text = "".join(data["train"]["document"])
ko_chars = sorted(list(set(ko_text)))
ko_vocab_size = len(ko_chars)
print(f"총 글자 수: {ko_vocab_size}")

총 글자 수: 2701


In [8]:
print(ko_chars[2000:2100])

['왓', '왔', '왕', '왜', '왠', '외', '왹', '왼', '요', '욕', '욘', '욜', '욤', '욥', '용', '우', '욱', '운', '욷', '울', '움', '웁', '웃', '웅', '워', '웍', '원', '월', '웜', '웠', '웡', '웨', '웬', '웰', '웸', '웹', '웻', '위', '윅', '윈', '윌', '윔', '윕', '윗', '윙', '유', '육', '윤', '율', '윱', '윳', '융', '으', '윽', '은', '을', '음', '읍', '읏', '응', '의', '읠', '이', '익', '인', '일', '읽', '잃', '임', '입', '잇', '있', '잉', '잊', '잎', '자', '작', '잔', '잖', '잘', '잠', '잡', '잣', '잤', '장', '잦', '재', '잭', '잰', '잼', '잽', '잿', '쟁', '쟈', '쟝', '쟤', '저', '적', '전', '절']


In [9]:
character_to_ids = {char:i for i, char in enumerate(ko_chars)}
ids_to_character = {i:char for i, char in enumerate(ko_chars)}

token_encode: Callable[[str], list[int]] = lambda s:[character_to_ids[c] for c in s]
token_decode: Callable[[list[int]], str] = lambda l: "".join([ids_to_character[i] for i in l])

print(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요."))
print(token_decode(token_encode("안녕하세요 함께 인공지능을 공부하게 되어 반가워요.")))

[1909, 1169, 2546, 1770, 2008, 0, 2551, 1061, 0, 2064, 977, 2157, 1209, 2055, 0, 977, 1658, 2546, 949, 0, 1283, 1942, 0, 1593, 908, 2024, 2008, 2]
안녕하세요 함께 인공지능을 공부하게 되어 반가워요.


In [10]:
tokenized_data = torch.tensor(token_encode(ko_text), dtype=torch.long)
print(tokenized_data.shape, tokenized_data.dtype)
print(tokenized_data[:100])

torch.Size([22162967]) torch.int64
tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987, 2555,    0, 2546, 1593,
        1028,    0, 2015, 1485,    0,  965, 2107, 2060,    0, 1617, 2465, 1542,
        2064,    0, 1808, 2273,    0, 2603, 1236, 1477,    0, 2037, 2555,    0,
        2263, 1430, 2055,    0, 1028, 2019, 2062, 1028, 1441,    0, 2562, 1841,
        1213, 1221,    2,    0, 2451, 2650,    0, 1808, 2273,    0, 2142, 1787,
        1028, 1950, 2060,    0, 1558, 1468, 1119,    0, 2555, 1787, 1477,    0,
        2037, 2555,    0, 1553, 1967, 1024, 2051,    0, 1015, 1541, 1477,    0,
           7,    3, 2117,    0, 2026,    0, 2062, 1740,    0, 2603, 1236, 2546,
         968,    0, 1558, 1468])


In [11]:
n = int(0.9 * len(tokenized_data))
train_dataset = tokenized_data[:n]
test_dataset = tokenized_data[n:]

In [12]:
block_size = 8
train_dataset[:block_size]

tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])

In [13]:
x = train_dataset[:block_size]
y = train_dataset[1:block_size+1]

for time in range(block_size):
    context = x[:time+1]
    target = y[time]

    print(f"입력 텐서: {context}")
    print(f"타깃 글자: {target}")

입력 텐서: tensor([1928])
타깃 글자: 2315
입력 텐서: tensor([1928, 2315])
타깃 글자: 0
입력 텐서: tensor([1928, 2315,    0])
타깃 글자: 2105
입력 텐서: tensor([1928, 2315,    0, 2105])
타깃 글자: 1658
입력 텐서: tensor([1928, 2315,    0, 2105, 1658])
타깃 글자: 908
입력 텐서: tensor([1928, 2315,    0, 2105, 1658,  908])
타깃 글자: 0
입력 텐서: tensor([1928, 2315,    0, 2105, 1658,  908,    0])
타깃 글자: 1987
입력 텐서: tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])
타깃 글자: 2555


In [14]:
torch.manual_seed(1234)

batch_size = 4
block_size = 8

def batch_function(mode: Literal["train", "test"]) -> tuple[torch.Tensor, torch.Tensor]:
    dataset = train_dataset if mode == "train" else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    return x, y

example_x, example_y = batch_function("train")
print(f"inputs: {example_x.shape}")
print("")
print("example_x의 실제 값")
print(example_x)
print("-----------------------")
print("targets : ", example_y.shape)
print("")
print("example_y의 실제 값")
print(example_y)
print("-----------------------")

for size in range(batch_size):
    for t in range(block_size):
        context = example_x[size, :t+1]
        target = example_y[size, t]
        print(f"input: {context}, target: {target}")
    print("-----------------------")
    print("-----------------------")

inputs: torch.Size([4, 8])

example_x의 실제 값
tensor([[1764, 2555,    0, 1236, 2248,    0, 2017, 1976],
        [   0, 1966, 2157,    0, 1951, 2062,    0, 2548],
        [   0, 1304, 1485, 1586,    0, 1907, 2450,    0],
        [   3,    2,    6,    5,    1,    0,    5,    3]])
-----------------------
targets :  torch.Size([4, 8])

example_y의 실제 값
tensor([[2555,    0, 1236, 2248,    0, 2017, 1976, 2546],
        [1966, 2157,    0, 1951, 2062,    0, 2548, 2289],
        [1304, 1485, 1586,    0, 1907, 2450,    0, 2480],
        [   2,    6,    5,    1,    0,    5,    3,    5]])
-----------------------
input: tensor([1764]), target: 2555
input: tensor([1764, 2555]), target: 0
input: tensor([1764, 2555,    0]), target: 1236
input: tensor([1764, 2555,    0, 1236]), target: 2248
input: tensor([1764, 2555,    0, 1236, 2248]), target: 0
input: tensor([1764, 2555,    0, 1236, 2248,    0]), target: 2017
input: tensor([1764, 2555,    0, 1236, 2248,    0, 2017]), target: 1976
input: tensor([1764, 25

In [16]:
class SemiGPT(nn.Module):
    def __init__(self, vocab_length: int):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        logits = self.embedding_token_table(inputs)
        return logits

model = SemiGPT(ko_vocab_size)
output = model(example_x, example_y)
print(output.shape)

torch.Size([4, 8, 2701])


In [18]:
class SemiGPT(nn.Module):
    def __init__(self, vocab_length: int):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        logits = self.embedding_token_table(inputs)
        batch, seq_length, vocab_length = logits.shape
        logits = logits.view(batch * seq_length, vocab_length)
        targets = targets.view(batch * seq_length)
        loss = F.cross_entropy(logits, targets)

        print(f"logits: {logits.shape}, targets: {targets.shape}")
        return logits, loss

model = SemiGPT(ko_vocab_size)
logits, loss = model(example_x, example_y)
print(loss)

logits: torch.Size([32, 2701]), targets: torch.Size([32])
tensor(8.5332, grad_fn=<NllLossBackward0>)


In [24]:
class SemiGPT(nn.Module):
    def __init__(self, vocab_length: int):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor=None) -> tuple[torch.Tensor, torch.Tensor]:
        logits = self.embedding_token_table(inputs)
        if targets is None:
            loss = None
        else:
            batch, seq_length, vocab_length = logits.shape
            logits = logits.view(batch * seq_length, vocab_length)
            targets = targets.view(batch*seq_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, inputs: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        for _ in range(max_new_tokens):
            logits, loss = self.forward(inputs)
            logits = logits[:, -1, :]
            print(f"logits: {logits.shape}")
            probs = F.softmax(logits, dim=-1)
            next_inputs = torch.multinomial(probs, num_samples=1)
            inputs = torch.cat([inputs, next_inputs], dim=-1)
        return inputs

model = SemiGPT(ko_vocab_size)
logits, loss = model(example_x, example_y)
print(loss)

token_decode(model.generate(torch.zeros((1,1),
                                        dtype=torch.long),
                            max_new_tokens=10)[0].tolist())

tensor(8.3806, grad_fn=<NllLossBackward0>)
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])
logits: torch.Size([1, 2701])


' 됨엷랲씌써엣퉈率湖깅'